# Create DTA Instance

Creates a new DTA entity in the DTA table.

**Sources:**
- `HISTORICAL` - From historical tsDTA processing (workflow_state=APPROVED)
- `UI` - From UI-initiated DTA creation (workflow_state=NOT_STARTED)

**What it does:**
1. Reads distinct trial metadata from silver table
2. Auto-generates DTA ID as UUID
3. Checks for MANUAL_REVIEW_REQUIRED records
4. Sets DTA status = MANUAL_REVIEW or ACTIVE accordingly
5. Creates DTA entity with audit columns
6. Creates DTA_WORKFLOW record for historical (auto-approved)


In [ ]:
# Cell 1: Imports
import json
import uuid
from datetime import datetime
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, TimestampType, BooleanType

print("="*80)
print("CREATE DTA INSTANCE")
print("="*80)


In [ ]:
# Cell 2: Get Configuration from Setup Task
globals_dict = json.loads(dbutils.jobs.taskValues.get(taskKey="setup", key="globals"))
versioning_dict = json.loads(dbutils.jobs.taskValues.get(taskKey="setup", key="versioning"))
created_by_principal = dbutils.jobs.taskValues.get(taskKey="setup", key="created_by_principal")
databricks_job_id = dbutils.jobs.taskValues.get(taskKey="setup", key="databricks_job_id")
databricks_job_name = dbutils.jobs.taskValues.get(taskKey="setup", key="databricks_job_name")
databricks_run_id = dbutils.jobs.taskValues.get(taskKey="setup", key="databricks_run_id")

catalog = globals_dict['catalog']
silver_schema = globals_dict['silver_schema']
gold_schema = globals_dict['gold_schema']

# Versioning configuration
versioning_enabled = versioning_dict.get('enabled', False)
registry_table_name = versioning_dict.get('registry_table', 'md_version_registry')
library_tables = versioning_dict.get('library_tables', [])

print(f"Catalog: {catalog}")
print(f"Silver Schema: {silver_schema}")
print(f"Gold Schema: {gold_schema}")
print(f"Versioning Enabled: {versioning_enabled}")
print(f"Created by: {created_by_principal}")


In [ ]:
# Cell 3: Get Widget Parameters
dbutils.widgets.text("source", "HISTORICAL", "Source")
dbutils.widgets.text("library_type", "", "Library Type (defaults from config)")
dbutils.widgets.text("trial_id", "", "Trial ID (optional)")
dbutils.widgets.text("data_stream_type", "", "Stream Type (optional)")
dbutils.widgets.text("data_provider_name", "", "Provider Name (optional)")

source = dbutils.widgets.get("source")
library_type_param = dbutils.widgets.get("library_type")
trial_id_param = dbutils.widgets.get("trial_id")
stream_type_param = dbutils.widgets.get("data_stream_type")
provider_param = dbutils.widgets.get("data_provider_name")

# Default to first library type from config if not specified
if library_type_param:
    library_type = library_type_param
elif library_tables:
    library_type = library_tables[0].get('library_type', 'transfer_variables')
else:
    library_type = 'transfer_variables'

# Get library table config for this library type
library_config = next((t for t in library_tables if t.get('library_type') == library_type), None)

print(f"\nSource: {source}")
print(f"Library Type: {library_type} (from {'param' if library_type_param else 'config'})")
print(f"Trial ID (param): {trial_id_param or 'auto-detect'}")
print(f"Stream Type (param): {stream_type_param or 'auto-detect'}")
print(f"Provider (param): {provider_param or 'auto-detect'}")


In [ ]:
# Cell 4: Define Table Schemas

def get_dta_schema():
    """
    =============================================================================
    SCHEMA: dta (Gold Layer - Data Transfer Agreement)
    =============================================================================
    This table stores the core metadata for each Trial-Specific Data Transfer 
    Agreement (tsDTA). Each record represents a unique data transfer contract 
    between J&J and a data provider for a specific trial and data stream.
    
    The table includes identifiers for the trial and data stream, references to 
    the parent document, the provider, the transfer type, and the current status 
    of the DTA (e.g., Draft, Review, Approved).
    
    Version metadata and audit timestamps support governance and traceability.
    This table is the authoritative record of all active and historical DTAs 
    used to drive ingestion rules and metadata validation.
    
    Version Pointers:
        - current_version_tag: Active version (draft or major) - what UI displays
        - current_draft_version: Latest draft in Silver (e.g., "1.0-DTA001-draft3")
        - latest_major_version: Latest approved in Gold (e.g., "1.0-DTA001-v1.0")
        - base_library_version: Library version branched from (e.g., "1.0")
    
    DTA Name Generation:
        - Historical import: {trial_id}_{data_stream_type}_{data_provider_name}
        - UI clone: <Change_this>_{trial_id}_{data_stream_type}_{data_provider_name}
    =============================================================================
    """
    return StructType([
        # Primary & Business Keys
        StructField("dta_id", StringType(), False),  # UUID primary key
        StructField("dta_number", StringType(), False),  # Sequential ID (DTA001, DTA002) for display
        StructField("dta_name", StringType(), True),  # User-friendly name for UI
        # Lineage
        StructField("parent_document_id", StringType(), True),  # FK to source tsDTA document
        # Business Key: trial_id + data_stream_type + data_provider_name
        StructField("trial_id", StringType(), True),  # Clinical trial ID (e.g., VAC18193RSV3001)
        StructField("data_stream_type", StringType(), True),  # Data category (Labs, Adverse Events)
        StructField("data_provider_name", StringType(), True),  # Vendor (LabCorp, Covance, PPD)
        # Status & Workflow
        StructField("status", StringType(), False),  # DRAFT, ACTIVE, MANUAL_REVIEW, ARCHIVED
        StructField("workflow_state", StringType(), False),  # NOT_STARTED, IN_REVIEW, APPROVED, REJECTED
        StructField("workflow_iteration", IntegerType(), True),  # Current approval cycle (1, 2, 3...)
        # Version Pointers
        StructField("current_version_tag", StringType(), True),  # Active version for UI display
        StructField("current_draft_version", StringType(), True),  # Latest draft in Silver
        StructField("latest_major_version", StringType(), True),  # Latest approved in Gold
        StructField("base_library_version", StringType(), True),  # Branched from library version
        StructField("notes", StringType(), True),  # General notes about the DTA
        # Audit columns
        StructField("created_by_principal", StringType(), True),  # User who created
        StructField("created_ts", TimestampType(), True),  # Creation timestamp
        StructField("last_updated_by_principal", StringType(), True),  # User who last modified
        StructField("last_updated_ts", TimestampType(), True),  # Last modification timestamp
        StructField("databricks_job_id", StringType(), True),  # Job ID for lineage
        StructField("databricks_job_name", StringType(), True),  # Job name for lineage
        StructField("databricks_run_id", StringType(), True)  # Run ID for lineage
    ])

def get_dta_workflow_schema():
    """
    =============================================================================
    SCHEMA: dta_workflow (Gold Layer - Workflow Tracking)
    =============================================================================
    This table tracks the approval lifecycle for DTAs across all stakeholders 
    involved in the metadata governance process.
    
    It models workflow steps between J&J reviewers, external data providers 
    (vendors), and J&J librarians, who are responsible for promoting finalized 
    metadata into the official library versions.
    
    The table captures workflow identifiers, statuses, and audit timestamps for 
    initiation and updates. It provides the structured sequence of approval 
    events required to move a DTA from Draft to Review to Approved to Published 
    Metadata Library.
    =============================================================================
    """
    return StructType([
        StructField("dta_workflow_id", StringType(), False),  # UUID primary key
        StructField("dta_id", StringType(), False),  # FK to dta table
        StructField("workflow_iteration", IntegerType(), False),  # Cycle number (1, 2, 3...)
        StructField("workflow_status", StringType(), False),  # NOT_STARTED, IN_REVIEW, APPROVED, REJECTED
        StructField("summary_comment", StringType(), True),  # Workflow notes/rejection reasons
        StructField("initiated_ts", TimestampType(), True),  # When workflow cycle started
        StructField("closed_ts", TimestampType(), True),  # When workflow cycle completed
        # Audit columns
        StructField("created_by_principal", StringType(), True),
        StructField("created_ts", TimestampType(), True),
        StructField("last_updated_by_principal", StringType(), True),
        StructField("last_updated_ts", TimestampType(), True),
        StructField("databricks_job_id", StringType(), True),
        StructField("databricks_job_name", StringType(), True),
        StructField("databricks_run_id", StringType(), True)
    ])

print("Schemas defined")


In [ ]:
# Cell 5: Configure Table Names (from config)
# Silver table based on library type
silver_table = f"{catalog}.{silver_schema}.md_transfer_variable_field_normalized"

# DTA tables
dta_table = f"{catalog}.{gold_schema}.dta"
dta_workflow_table = f"{catalog}.{gold_schema}.dta_workflow"

# Library table from config
library_table_name = library_config.get('name', 'md_transfer_variables_library') if library_config else 'md_transfer_variables_library'
library_table = f"{catalog}.{gold_schema}.{library_table_name}"

# Registry table from config
registry_table = f"{catalog}.{gold_schema}.{registry_table_name}"

print(f"\nSilver Source: {silver_table}")
print(f"DTA Table: {dta_table}")
print(f"DTA Workflow Table: {dta_workflow_table}")
print(f"Library Table: {library_table}")
print(f"Registry Table: {registry_table}")


In [ ]:
# Cell 6: Read Silver Data and Get Distinct DTAs
print(f"\n{'='*80}")
print("Reading silver data for DTA metadata...")
print(f"{'='*80}")

# Read silver table
df_silver = spark.table(silver_table)

# Get distinct DTA contexts
# For HISTORICAL: Each parent_document_id = separate DTA (one DTA per uploaded file)
# For UI: Group by trial/stream/provider (one DTA per logical context)
if source == "HISTORICAL":
    # Include parent_document_id to create one DTA per uploaded file
    df_dta_contexts = df_silver.select(
        "parent_document_id",
        "trial_id", 
        "data_stream_type", 
        "data_provider_name"
    ).distinct()
    print("  Mode: HISTORICAL - One DTA per uploaded file (parent_document_id)")
else:
    # Group by trial context only (UI mode)
    df_dta_contexts = df_silver.select(
        "trial_id", 
        "data_stream_type", 
        "data_provider_name"
    ).distinct().withColumn("parent_document_id", F.lit(None))
    print("  Mode: UI - One DTA per trial/stream/provider context")

# If parameters provided, filter to specific context
if trial_id_param:
    df_dta_contexts = df_dta_contexts.filter(F.col("trial_id") == trial_id_param)
if stream_type_param:
    df_dta_contexts = df_dta_contexts.filter(F.col("data_stream_type") == stream_type_param)
if provider_param:
    df_dta_contexts = df_dta_contexts.filter(F.col("data_provider_name") == provider_param)

dta_contexts = df_dta_contexts.collect()
print(f"Found {len(dta_contexts)} distinct DTA context(s) to process")


In [ ]:
# Cell 7: Ensure Tables Exist
now = datetime.now()

# Create DTA table if not exists
if not spark.catalog.tableExists(dta_table):
    print(f"Creating DTA table: {dta_table}")
    empty_df = spark.createDataFrame([], schema=get_dta_schema())
    empty_df.write.format("delta").mode("overwrite").saveAsTable(dta_table)
    print(f"Created {dta_table}")
else:
    print(f"DTA table exists: {dta_table}")

# Create DTA_WORKFLOW table if not exists
if not spark.catalog.tableExists(dta_workflow_table):
    print(f"Creating DTA_WORKFLOW table: {dta_workflow_table}")
    empty_df = spark.createDataFrame([], schema=get_dta_workflow_schema())
    empty_df.write.format("delta").mode("overwrite").saveAsTable(dta_workflow_table)
    print(f"Created {dta_workflow_table}")
else:
    print(f"DTA_WORKFLOW table exists: {dta_workflow_table}")


In [ ]:
# Cell 8: Create DTA Instances
print(f"\n{'='*80}")
print("Creating DTA instances...")
print(f"{'='*80}")

# Get the current max DTA number from existing table
def get_next_dta_number(spark, dta_table, offset=0):
    """Get the next sequential DTA number (DTA001, DTA002, etc.)"""
    try:
        # Check if table exists first
        if not spark.catalog.tableExists(dta_table):
            print(f"  ⚠️ DTA table does not exist yet, starting from DTA001")
            return f"DTA{(1 + offset):03d}"
        
        # Query for max number
        result = spark.sql(f"""
            SELECT MAX(CAST(REGEXP_EXTRACT(dta_number, 'DTA([0-9]+)', 1) AS INT)) as max_num
            FROM {dta_table}
        """).first()
        
        max_num = result.max_num if result and result.max_num is not None else 0
        next_num = max_num + 1 + offset
        
        print(f"  📊 Current max DTA number: {max_num}, Next: DTA{next_num:03d}")
        return f"DTA{next_num:03d}"
        
    except Exception as e:
        print(f"  ❌ ERROR getting DTA number: {e}")
        print(f"  ⚠️ Defaulting to DTA{(1 + offset):03d}")
        return f"DTA{(1 + offset):03d}"

dta_records = []
workflow_records = []
created_dta_ids = []
created_dta_numbers = []  # Track DTA numbers for version tag

for idx, ctx in enumerate(dta_contexts):
    parent_document_id = ctx["parent_document_id"]  # Will be None for UI mode
    trial_id = ctx["trial_id"]
    stream_type = ctx["data_stream_type"]
    provider = ctx["data_provider_name"]
    
    if parent_document_id:
        print(f"\nProcessing: {trial_id} / {stream_type} / {provider} (Doc: {parent_document_id[:8]}...)")
    else:
        print(f"\nProcessing: {trial_id} / {stream_type} / {provider}")
    
    # Build filter condition - for HISTORICAL, filter by parent_document_id
    if source == "HISTORICAL" and parent_document_id:
        filter_condition = (F.col("parent_document_id") == parent_document_id)
    else:
        filter_condition = (
            (F.col("trial_id") == trial_id) &
            (F.col("data_stream_type") == stream_type) &
            (F.col("data_provider_name") == provider)
        )
    
    # Check for MANUAL_REVIEW_REQUIRED records
    review_count = df_silver.filter(
        filter_condition &
        (F.col("row_status") == "MANUAL_REVIEW_REQUIRED")
    ).count()
    
    total_count = df_silver.filter(filter_condition).count()
    
    needs_review = review_count > 0
    print(f"  Total records: {total_count}, Needs review: {review_count}")
    
    # Generate UUID for DTA
    dta_id = str(uuid.uuid4())
    
    # Generate sequential DTA number (DTA001, DTA002, etc.)
    dta_number = get_next_dta_number(spark, dta_table, offset=idx)
    
    created_dta_ids.append(dta_id)
    created_dta_numbers.append(dta_number)
    
    # Determine status, workflow_state, and version pointers based on source
    # Base library version - what this DTA branches from
    base_library_version = "1.0"  # TODO: Could query latest LIBRARY_MAJOR from registry
    
    if source == "HISTORICAL":
        # Historical: Already approved, will go directly to DTA_MAJOR
        status = "MANUAL_REVIEW" if needs_review else "ACTIVE"
        workflow_state = "APPROVED"
        notes = f"Created from historical tsDTA data. Records needing review: {review_count}/{total_count}"
        # Version pointers for HISTORICAL (will be set after DTA_MAJOR creation)
        current_draft_version = None  # No draft phase for historical
        latest_major_version = None  # Will be set by create_historical_dta_major
        current_version_tag = None  # Will be set by create_historical_dta_major
    else:  # UI
        # UI: Starts as draft in Silver
        status = "DRAFT"
        workflow_state = "NOT_STARTED"
        notes = "Created from UI"
        # Version pointers for UI (draft starts in Silver)
        draft_version_tag = f"{base_library_version}-{dta_number}-draft1"
        current_draft_version = draft_version_tag
        latest_major_version = None  # No approved version yet
        current_version_tag = draft_version_tag  # Points to Silver draft
    
    # Create DTA record
    dta_record = {
        "dta_id": dta_id,
        "dta_number": dta_number,  # Sequential DTA number for version tags
        "parent_document_id": parent_document_id,  # Link to source document (for historical lineage)
        "trial_id": trial_id,
        "data_stream_type": stream_type,
        "data_provider_name": provider,
        "status": status,
        "workflow_state": workflow_state,
        "workflow_iteration": 1 if source == "HISTORICAL" else 0,
        # Version Pointers
        "current_version_tag": current_version_tag,
        "current_draft_version": current_draft_version,
        "latest_major_version": latest_major_version,
        "base_library_version": base_library_version,
        "notes": notes,
        "created_by_principal": created_by_principal,
        "created_ts": now,
        "last_updated_by_principal": created_by_principal,
        "last_updated_ts": now,
        "databricks_job_id": databricks_job_id,
        "databricks_job_name": databricks_job_name,
        "databricks_run_id": databricks_run_id
    }
    dta_records.append(dta_record)
    
    # For HISTORICAL, create auto-approved workflow record
    if source == "HISTORICAL":
        workflow_record = {
            "dta_workflow_id": str(uuid.uuid4()),
            "dta_id": dta_id,
            "workflow_iteration": 1,
            "workflow_status": "APPROVED",
            "summary_comment": "Historical DTA - auto-approved during import",
            "initiated_ts": now,
            "closed_ts": now,
            "created_by_principal": created_by_principal,
            "created_ts": now,
            "last_updated_by_principal": created_by_principal,
            "last_updated_ts": now,
            "databricks_job_id": databricks_job_id,
            "databricks_job_name": databricks_job_name,
            "databricks_run_id": databricks_run_id
        }
        workflow_records.append(workflow_record)
    
    # Generate user-friendly DTA name
    # Historical: {trial_id}_{data_stream_type}_{data_provider_name}
    # UI: <Change_this>_{trial_id}_{data_stream_type}_{data_provider_name}
    if source == "HISTORICAL":
        dta_name = f"{trial_id}_{stream_type}_{provider}"
    else:
        dta_name = f"<Change_this>_{trial_id}_{stream_type}_{provider}"
    
    # Add dta_name to the record
    dta_record["dta_name"] = dta_name
    
    print(f"  Created DTA: {dta_number} (ID: {dta_id[:8]}...)")
    print(f"  Name: {dta_name}")
    print(f"  Status: {status}, Workflow State: {workflow_state}")


In [ ]:
# Cell 9: Save to Tables
print(f"\n{'='*80}")
print("Saving DTA records...")
print(f"{'='*80}")

if dta_records:
    # Save DTA records (with schema evolution for new columns like parent_document_id)
    dta_df = spark.createDataFrame(dta_records, schema=get_dta_schema())
    dta_df.write.format("delta").mode("append").option("mergeSchema", "true").saveAsTable(dta_table)
    print(f"Saved {len(dta_records)} DTA record(s)")

if workflow_records:
    # Save workflow records
    wf_df = spark.createDataFrame(workflow_records, schema=get_dta_workflow_schema())
    wf_df.write.format("delta").mode("append").saveAsTable(dta_workflow_table)
    print(f"Saved {len(workflow_records)} workflow record(s)")


In [ ]:
# Cell 10: Set Task Values for Downstream Tasks
# Store created DTA IDs and numbers for downstream processing
dbutils.jobs.taskValues.set(key="created_dta_ids", value=json.dumps(created_dta_ids))
dbutils.jobs.taskValues.set(key="created_dta_numbers", value=json.dumps(created_dta_numbers))
dbutils.jobs.taskValues.set(key="source", value=source)
dbutils.jobs.taskValues.set(key="library_type", value=library_type)
dbutils.jobs.taskValues.set(key="library_table", value=library_table)
dbutils.jobs.taskValues.set(key="registry_table", value=registry_table)
dbutils.jobs.taskValues.set(key="dta_count", value=len(created_dta_ids))

print(f"\n{'='*80}")
print("Summary")
print(f"{'='*80}")
print(f"Source: {source}")
print(f"Library Type: {library_type}")
print(f"DTAs created: {len(created_dta_ids)}")
for dta_id, dta_num in zip(created_dta_ids, created_dta_numbers):
    print(f"  - {dta_num} (ID: {dta_id[:8]}...)")
